# Noisy 1B5 (Finetune Training)
We will be finetuning a 1B5 model, and intentionally overtrain it.
This is to measure how the introduction of a noisy state will impact train.

The goal of this experiment, is to measure two seperate objective
- test for overfitting conditions across multiple epoch
- test if overfitting is mitigated with the introduction of a noisy state

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

In [2]:
# First lets get the RWKV base model which we will be finetuning
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/
!cd ../../../model/ && wget https://huggingface.co/BlinkDL/rwkv-4-pileplus/resolve/main/RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth

--2023-07-18 15:52:11--  https://huggingface.co/BlinkDL/rwkv-4-pileplus/resolve/main/RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth
Resolving huggingface.co (huggingface.co)... 13.224.249.10, 13.224.249.119, 13.224.249.44, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a0/d0/a0d028cbd69f2f788f85a9e3b66d39102ab8e09500502ba8b9780cd52652da12/d3ca6ee3a10b4586b2dff826e3c99dbfa28ecaa71596e82e59a1948c38029e40?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth%3B+filename%3D%22RWKV-4-PilePlus-1B5-20230520-2942-486Gtokens-ctx4096.pth%22%3B&Expires=1689925931&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTkyNTkzMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hMC9kMC9hMGQwMjhjYmQ2OWYyZjc4OGY4NWE5ZTNiNjZkMzkxMDJhYjhlMD

In [3]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="MultiEpoch-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment/notebook/experiment/noisy-training
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo
PROJECT_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment


In [4]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-baseline.yaml"

Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 41.58it/s]


## Stage 1 : Baseline model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-baseline.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct Tune Baseline (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Baseline/last.ckpt" "../model/MultiEpoch-1B5-Baseline.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Baseline.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B4-Baseline/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/MultiEpoch-1B4-Baseline.pth
-rw-r--r-- 1 root root 5.3G Jul 18 13:12 ../model/MultiEpoch-1B4-Baseline.pth


## Stage 2 : Noisy model training

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B5-instruct-noisy.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct Tune Noisy (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B5-Noisy/last.ckpt" "../model/MultiEpoch-1B5-Noisy.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B5-Noisy.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B4-Baseline/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/MultiEpoch-1B4-Baseline.pth
-rw-r--r-- 1 root root 5.3G Jul 18 13:12 ../model/MultiEpoch-1B4-Baseline.pth
